# Layout

All UBC PDK cells are conveniently combined into the PDK class.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pp
from ubc.pdk import PDK

In [ ]:
PDK.get_factory_names()    

## Leaf Components

You can see the most basic leaf components from the UBC PDK

In [ ]:
PDK.bend_euler()

In [ ]:
PDK.crossing()

In [ ]:
PDK.dc_adiabatic()

In [ ]:
PDK.dc_broadband_te()

In [ ]:
PDK.dc_broadband_tm()

In [ ]:
PDK.y_splitter()

In [ ]:
PDK.y_adiabatic()

## Compounded Components 

You can define compounded components as a combination of several leaf components

In [ ]:
splitter=PDK.dc_broadband_te
c = PDK.mzi(splitter=splitter, combiner=splitter)
c

In [ ]:
PDK.ring_single()

In [ ]:
PDK.dbr()

In [ ]:
PDK.spiral()

## Components with grating couplers

To test your devices you can add grating couplers. Both for single fibers and for fiber arrays.

In [ ]:
component = PDK.mzi()
component_single_fiber = PDK.add_fiber_single(component=component, grating_coupler=PDK.grating_coupler_te1550)
component_single_fiber

In [ ]:
component = PDK.mzi()
component_fiber_array = PDK.add_fiber_array(component=component, grating_coupler=PDK.grating_coupler_te1550)
component_fiber_array